In [1]:
import ccxt, pandas as pd, matplotlib.pyplot as plt, sys, indicateur_techniques as ind
sys.path.append('../Ohlcvplus')
from ohlcv import OhlcvPlus

In [4]:
class Backtest:
    def __init__(self, capital):
        self.capital = capital
        self.position = None

        self.all_position = pd.DataFrame({
			"open_pos" : [],
            "close_pos" : [],
			"mode" : [],
			"open" : [],
			"close" : []
		})

        self.stop_loss = None
        self.take_profit = None

    def load_data(self, symbol='BTC/USDT', time='30m', length=500, sinces='2023-01-01 00:00:00'):
		#telecharcgement des donnée ohlcv
        ohlcvp = OhlcvPlus(ccxt.binance(), database_path='data.db')
        self.data = ohlcvp.load(market=symbol, timeframe=time, since=sinces, limit=length, update=True, verbose=True, workers=100)
        self.close = self.data.close
    
    def copie_data(self, df):
        self.data = df
        self.close = self.data.close
    
    def open_pos(self, taille:int, pos:int, take_profit=None, stop_loss=None):
        if self.position is None:
            self.position = [self.close[pos], taille / self.close[pos], pos]
            print("open", pos)

            if take_profit:
                self.take_profit = self.close[pos] * (1 + (take_profit / 100))
            
            if stop_loss:
                self.stop_loss = self.close[pos] * (1 - (stop_loss / 100))
    
    def close_pos(self, pos:int):
        if self.position is not None:
            self.capital += (self.close[pos] - self.position[0]) * self.position[1]
            print('close', pos)
            print(self.capital)

            add_ligne = pd.DataFrame({
                    "open_pos" : [self.position[2]],
                    "close_pos" : [pos],
					"mode" : ["long"],
					"open" : [self.position[0]],
					"close" : [self.close[pos]]
				})
            self.all_position = pd.concat([self.all_position, add_ligne])
            self.position = None
    
    def graphique(self, *args:list):
        plt.close()
        if len(args) > 1:
            fig, axs = plt.subplots(len(args))
            axs[0].plot(self.close)
            for i in range(len(args)):
                for ind in args[i]:
                    axs[i].plot(ind)
            
            for row in range(len(self.all_position)):
                x = self.all_position['open_pos'].iloc[row]
                y = self.close[x]
                axs[0].plot(x, y, marker='^', markersize=10, color='g', label='Triangles')
                
                x = self.all_position['close_pos'].iloc[row]
                y = self.close[x]
                axs[0].plot(x, y, marker='v', markersize=10, color='r', label='Triangles')

        else: 
            fig, axs = plt.subplots()
            axs.plot(self.close)
            for ind in args[0]:
                axs.plot(ind)
            for row in range(len(self.all_position)):
                x = self.all_position['open_pos'].iloc[row]
                y = self.close[x]
                axs.plot(x, y, marker='^', markersize=10, color='g', label='Triangles')
                
                x = self.all_position['close_pos'].iloc[row]
                y = self.close[x]
                axs.plot(x, y, marker='v', markersize=10, color='r', label='Triangles')


    def trier_signal(self, series):
        result = []
        current_value = False
        for value in series:
            if value == current_value:
                result.append(False)
            else:
                current_value = value
                result.append(value)
        result[0] = False
        return result
    
    def updates(self, pos):
        if self.position is not None:
            self.stop_loss_take_profit(pos)
    
    def stop_loss_take_profit(self, pos):
        if self.take_profit:
            if self.take_profit < self.close[pos]:
                self.close_pos(pos)
                self.take_profit = None
        if self.stop_loss:
            if self.stop_loss > self.close[pos]:
                self.close_pos(pos)
                self.stop_loss = None


In [97]:
%matplotlib
test = Backtest(1000)
test.copie_data(pd.read_csv('data_btc.csv'))
test.data['achat'] = test.trier_signal(test.close > ind.mm(test.close, 2000))
test.data['vente'] = test.close < ind.mm(test.close, 2000)

Using matplotlib backend: TkAgg


In [98]:
for i in range(len(test.data)):
    if test.data.achat[i] == True:
        test.open_pos(5, i)
    elif test.data.vente[i] == True:
        test.close_pos(i)

open 1999
close 2932
1000.2530890476961
open 3438
close 5318
1001.1153925485221
open 5514
close 5559
1001.0536769221142
open 5561
close 5565
1001.0334153482576
open 5566
close 5763
1001.05537115194
open 5764
close 5765
1001.0496385591147
open 5766
close 5768
1001.0419077642805
open 5773
close 5779
1001.0297794899437
open 5784
close 5787
1001.0094620329593
open 5843
close 5857
1000.9955029404472
open 5869
close 5877
1000.9908648318025
open 5878
close 5879
1000.9749976696515
open 5898
close 6028
1000.9389973035637
open 6032
close 6095
1000.8930276187073
open 7100
close 7133
1000.8418735670599
open 7158
close 7160
1000.8315196679528
open 7170
close 7180
1000.8214865896987
open 7190
close 7193
1000.8147455800796
open 7194
close 7195
1000.7966990678498
open 7202
close 7203
1000.7859444033288
open 8148
close 8149
1000.7138301811261
open 8152
close 8153
1000.7058762296112
open 8155
close 8188
1000.687165111494
open 8189
close 9811
1001.1348891592909
